In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf

from glob import glob
from scipy.io import loadmat
from tensorflow import keras
from tensorflow.keras import layers

In [96]:
IMAGE_SIZE = 64
BATCH_SIZE = 4
NUM_CLASSES = 32
NUM_TRAIN_IMAGES = 1000
NUM_VAL_IMAGES = 50

In [76]:
train_images = sorted(glob('../data/coarse_tuning/leftImg8bit/train/**/*.png',
                           recursive=True))[:NUM_TRAIN_IMAGES]
train_masks = sorted(glob('../data/finetuning/gtFine/train/**/*octogroups.png',
                          recursive=True))[:NUM_TRAIN_IMAGES]
val_images = sorted(glob('../data/coarse_tuning/leftImg8bit/val/**/*.png',
                         recursive=True))[:NUM_VAL_IMAGES]
val_masks = sorted(glob('../data/finetuning/gtFine/val/**/*octogroups.png',
                        recursive=True))[:NUM_VAL_IMAGES]

In [77]:
print('Found', len(train_images), 'training images')
print('Found', len(train_masks), 'training masks')
print('Found', len(val_images), 'validation images')
print('Found', len(val_masks), 'validation masks')

Found 1000 training images
Found 1000 training masks
Found 50 validation images
Found 50 validation masks


In [78]:
def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.image.decode_png(image, channels=1)
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    else:
        image = tf.image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = image / 127.5 - 1
    return image

In [79]:
def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask

In [80]:
def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

In [81]:
train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

In [82]:
print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

Train Dataset: <BatchDataset shapes: ((4, 64, 64, 3), (4, 64, 64, 1)), types: (tf.float32, tf.float32)>
Val Dataset: <BatchDataset shapes: ((4, 64, 64, 3), (4, 64, 64, 1)), types: (tf.float32, tf.float32)>


In [83]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding=padding,
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)

In [84]:
def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output


In [85]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)

In [86]:
model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
#model.summary()

In [87]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [99]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=loss,
    metrics=["accuracy"],
    steps_per_execution=250,
)

In [98]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=2)

Epoch 1/2
250/250 [==============================] - 20s 82ms/step - loss: nan - accuracy: 1.4917e-04 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/2
250/250 [==============================] - 13s 51ms/step - loss: nan - accuracy: 1.4917e-04 - val_loss: nan - val_accuracy: 0.0000e+00
